In [68]:
import neurokit2 as nk
import numpy as np
from scipy.signal import find_peaks

In [86]:
import neurokit2 as nk
import numpy as np

def extract_features(ecg_signal, fs):
    # 1) Run NeuroKit2 processing
    signals, info = nk.ecg_process(ecg_signal, sampling_rate=fs)
    features = nk.ecg_analyze(signals, sampling_rate=fs)

    # 2) Start with main HRV and HR metrics
    summary = {
        "ECG_Rate_Mean":  float(features.get("ECG_Rate_Mean", 0)),
        "HRV_SDNN":       float(features.get("HRV_SDNN", 0)),
        "HRV_RMSSD":      float(features.get("HRV_RMSSD", 0)),
        "HRV_pNN50":      float(features.get("HRV_pNN50", 0)),
    }

    # 3) Manual calculation of QRS duration and QT interval
    q = signals.index[signals["ECG_Q_Peaks"] == 1].to_numpy()
    s = signals.index[signals["ECG_S_Peaks"] == 1].to_numpy()
    t = signals.index[signals["ECG_T_Offsets"] == 1].to_numpy()

    min_len = min(len(q), len(s))
    qrs = ((s[:min_len] - q[:min_len]) / fs * 1000)
    min_len_qt = min(len(q), len(t))
    qt = ((t[:min_len_qt] - q[:min_len_qt]) / fs * 1000)

    summary["ECG_QRS_Duration"] = float(np.mean(qrs)) if qrs.size > 0 else np.nan
    summary["ECG_QT_Interval"]  = float(np.mean(qt))  if qt.size > 0  else np.nan

    return summary


In [87]:
# Load your cleaned fetal ECG
fecg = np.load("ica_cleaned_signals/sub01_l1_c0_combined_cleaned_fecg.npy")
fs   = 250  # Confirm your dataset’s sampling rate

features = extract_features(fecg, fs)
for name, value in features.items():
    print(f"{name}: {value:.2f}")

ECG_Rate_Mean: 143.02
HRV_SDNN: 9.30
HRV_RMSSD: 13.31
HRV_pNN50: 0.14
ECG_QRS_Duration: 69.74
ECG_QT_Interval: 154.64


/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/1494222083.py:11: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "ECG_Rate_Mean":  float(features.get("ECG_Rate_Mean", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/1494222083.py:12: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "HRV_SDNN":       float(features.get("HRV_SDNN", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/1494222083.py:13: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "HRV_RMSSD":      float(features.get("HRV_RMSSD", 0)),
/var/folders/25/rb9t435n4mzgnk1l82dg2c1c0000gn/T/ipykernel_10912/1494222083.py:14: FutureWarning: Calling float on a single element Series is deprec